In [1]:
import random

def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

def egcd(a, b):
    if b == 0:
        return a, 1, 0
    else:
        d, x, y = egcd(b, a % b)
        return d, y, x - (a // b) * y
    
def modular_linear_equation_solver(a, b, n):
    ret = []
    d, x, y = egcd(a, n)
    if b % d == 0:
        x_0 = (x * (b / d)) % n
        for i in range(0, d):
            ret.append((x_0 + i * (n / d)) % n)
    return ret

def mul_inv(a, n):
    d, x, y = egcd(a, n)
    if d == 1:
        return x % n
    else:
        raise Exception('gcd(a, n) != 1')
        
def crt(list_a, list_n):
    a, p = 0, 1
    for n in list_n:
        p = p * n
    for i in range(len(list_a)):
        a_i, n_i = list_a[i], list_n[i]
        m_i = p / n_i
        a += (a_i * m_i * mul_inv(m_i, n_i))
    return a % p

def modular_exponentiation(a, b, n):
    c = 0
    d = 1
    b_binary = "{0:b}".format(b)
    for i in range(0, len(b_binary)):
        c = 2 * c
        d = (d * d) % n
        if b_binary[i] == "1":
            c = c + 1
            d = (d * a) % n
    return d

def generate_keys(p, q):
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = random.randrange(1, phi_n, 2)
    while gcd(e, phi_n) != 1:
        e = random.randrange(1, phi_n, 2)
    d = mul_inv(e, phi_n)
    P, S = (e, n), (d, n)
    return P, S

def encrypt(P, plain_text):
    e, n = P
    cipher_text = []
    for c in plain_text:
        cipher_text.append(modular_exponentiation(ord(c), e, n))
    return cipher_text

def decrypt(S, cipher_text):
    d, n = S
    plain_text = []
    for c in cipher_text:
        plain_text.append(chr(modular_exponentiation(c, d, n)))
    return "".join(plain_text)

In [5]:
p = 13373264125643791
q = 18214494739464977
P, S = generate_keys(p, q)
print("public key = {0}".format(P))
print("private key = {0}".format(S))

public key = (188051962538484167904353720798117, 243587249066014526390865022007807)
private key = (10483428855909982076734674343853, 243587249066014526390865022007807)


In [6]:
M = "Hello World!"
cipher_text = encrypt(P, M)
print(cipher_text)

[230164026490259542266727941843156, 213203598977599812031652931867490, 197861446314166194165833898662120, 197861446314166194165833898662120, 87618811313803517432383999882379, 91283656275970604969822382558752, 208569279713122133373955925940502, 87618811313803517432383999882379, 69360961346398362019067480074514, 197861446314166194165833898662120, 183253257905709808413112550200746, 93971463651636519089437471584153]


In [7]:
plain_text = decrypt(S, cipher_text)
print(plain_text)

Hello World!
